In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sqlite3

In [2]:
!pwd

/Users/carlosgovea/Documents/Flatiron/DSNYC103122/Phase-1-project-template/notebooks


In [18]:
#Only need these 3 df as will be focusing on Sale distribution

m_gross = pd.read_csv('/Users/carlosgovea/Documents/Flatiron/DSNYC103122/Phase-1-project-template/zippedData/bom.movie_gross.csv')
movie_info = pd.read_csv('/Users/carlosgovea/Documents/Flatiron/DSNYC103122/Phase-1-project-template/zippedData/rt.movie_info.tsv', delimiter ='\t')
movie_budget = pd.read_csv('/Users/carlosgovea/Documents/Flatiron/DSNYC103122/Phase-1-project-template/zippedData/tn.movie_budgets.csv')
popular = pd.read_csv('/Users/carlosgovea/Documents/Flatiron/DSNYC103122/Phase-1-project-template/zippedData/tmdb.movies.csv', index_col =0)

In [4]:
#BEGIN THE SCRUB - Changing Dtypes to appropriate 
movie_info['theater_date'] = pd.to_datetime(movie_info['theater_date'])
movie_info['release_year'] = movie_info['theater_date'].dt.year
movie_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            1560 non-null   int64         
 1   synopsis      1498 non-null   object        
 2   rating        1557 non-null   object        
 3   genre         1552 non-null   object        
 4   director      1361 non-null   object        
 5   writer        1111 non-null   object        
 6   theater_date  1201 non-null   datetime64[ns]
 7   dvd_date      1201 non-null   object        
 8   currency      340 non-null    object        
 9   box_office    340 non-null    object        
 10  runtime       1530 non-null   object        
 11  studio        494 non-null    object        
 12  release_year  1201 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(10)
memory usage: 158.6+ KB


In [6]:
#Parsing through movie info to only include movies from 2010 forward

relevent_movie_info= movie_info.loc[movie_info['release_year']>2010,:]
relevent_movie_info.drop(['currency'], axis=1, inplace =True) #dropped currency because too many nulls
relevent_movie_info['box_office'] = pd.to_numeric(movie_info['box_office'], errors ='coerce') 
relevent_movie_info.drop(['box_office'], axis =1, inplace= True)
relevent_movie_info

/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-6-d8c9ca56fe41>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevent_movie_info['box_office'] = pd.to_numeric(movie_info['box_office'], errors ='coerce')


,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,runtime,studio,release_year
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,2012-08-17,"Jan 1, 2013",108 minutes,Entertainment One,2012.0
16,23,A fictional film set in the alluring world of ...,R,Drama,NaN,NaN,2013-12-20,"Mar 18, 2014",129 minutes,Sony Pictures,2013.0
18,25,"From ancient Japan's most enduring tale, the e...",PG-13,Action and Adventure|Drama|Science Fiction and...,Carl Erik Rinsch,Chris Morgan|Hossein Amini,2013-12-25,"Apr 1, 2014",127 minutes,Universal Pictures,2013.0
41,54,Journalist Jep Gambardella (the dazzling Toni ...,NR,Comedy|Drama,Paolo Sorrentino,Paolo Sorrentino|Umberto Contarello,2013-11-15,"Mar 25, 2014",142 minutes,Janus Films,2013.0
44,57,Life doesn't always go according to plan. Pat ...,R,Comedy,NaN,NaN,2012-11-21,"Apr 30, 2013",122 minutes,The Weinstein Company,2012.0
...,...,...,...,...,...,...,...,...,...,...,...
1524,1960,SELMA is the story of a movement. The film chr...,PG-13,Drama,Ava DuVernay,Paul Webb,2015-01-09,"May 5, 2015",127 minutes,Paramount Pictures,2015.0
1530,1968,"This holiday season, acclaimed filmmaker Camer...",PG,Comedy|Drama,Cameron Crowe,Aline Brosh McKenna|Cameron Crowe,2011-12-23,"Apr 3, 2012",126 minutes,20th Century Fox,2011.0
1536,1975,The year is 1953. A visiting archaeologist cal...,NR,Drama|Romance,Vikramaditya Motwane,Vikramaditya Motwane|Bhavani Iyer|Anurag Kashyap,2013-07-03,"Jan 12, 2015",142 minutes,NaN,2013.0
1537,1976,"Embrace of the Serpent features the encounter,...",NR,Action and Adventure|Art House and International,Ciro Guerra,Ciro Guerra|Jacques Toulemonde Vidal,2016-02-17,"Jun 21, 2016",123 minutes,Buffalo Films,2016.0


In [7]:
#CLEANING MOVIE BUDGET 
movie_budget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [8]:
movie_budget['release_date'] # needs to be changed to date-time data type

0       Dec 18, 2009
1       May 20, 2011
2        Jun 7, 2019
3        May 1, 2015
4       Dec 15, 2017
            ...     
5777    Dec 31, 2018
5778     Apr 2, 1999
5779    Jul 13, 2005
5780    Sep 29, 2015
5781     Aug 5, 2005
Name: release_date, Length: 5782, dtype: object

In [9]:
#Code below switches release year to date time for release year and into new column year

movie_budget['release_date'] =pd.to_datetime(movie_budget['release_date'])
movie_budget['release_year'] = movie_budget['release_date'].dt.year # made new column only showing year as int

# Check for duplicated/ irrelevent data
movie_budget[movie_budget.duplicated(subset=['movie', 'release_year'])]

# The only movie that is duplicated within same year and name are diff entries

movie_budget.loc[movie_budget['movie']=='Home'] 


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year
243,44,2015-03-27,Home,"$130,000,000","$177,397,510","$385,997,896",2015
3455,56,2009-06-05,Home,"$12,000,000",$0,$0,2009
5459,60,2009-04-23,Home,"$500,000","$15,433","$44,793,168",2009


In [10]:
#Changing Production, Domestic, and worldwide into floats instead of strings... cant decide if $ symbol will pose a problem

movie_budget['production_budget'] = movie_budget['production_budget'].replace('[\$,]', '', regex=True).astype(float)
movie_budget['domestic_gross'] = movie_budget['domestic_gross'].replace('[\$,]', '',regex=True).astype(float)
movie_budget['worldwide_gross'] = movie_budget['worldwide_gross'].replace('[\$,]','', regex= True).astype(float)

# only going to be using movies from 2010 forward

relevent_movies_10 =movie_budget.loc[movie_budget['release_year']>2010,:]

relevent_movies_10.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1920 entries, 1 to 5780
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 1920 non-null   int64         
 1   release_date       1920 non-null   datetime64[ns]
 2   movie              1920 non-null   object        
 3   production_budget  1920 non-null   float64       
 4   domestic_gross     1920 non-null   float64       
 5   worldwide_gross    1920 non-null   float64       
 6   release_year       1920 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 120.0+ KB


In [12]:
#CLEANING SALES DF 
m_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [13]:
m_gross[m_gross['foreign_gross'].isna()]

,title,studio,domestic_gross,foreign_gross,year
222,Flipped,WB,1800000.0,NaN,2010
254,The Polar Express (IMAX re-issue 2010),WB,673000.0,NaN,2010
267,Tiny Furniture,IFC,392000.0,NaN,2010
269,Grease (Sing-a-Long re-issue),Par.,366000.0,NaN,2010
280,Last Train Home,Zeit.,288000.0,NaN,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


In [14]:
#THIS CHANGES FOREIGN GROSS TO NUMERIC DESPITE THE OBJECT NaNs

m_gross['foreign_gross'] = pd.to_numeric(m_gross['foreign_gross'], errors = 'coerce') # Will need some guidence here on coerce 
m_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2032 non-null   float64
 4   year            3387 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 132.4+ KB


In [15]:
#checking for duplicates for m_gross
m_gross[m_gross.duplicated(subset =['title'])]

,title,studio,domestic_gross,foreign_gross,year
3045,Bluebeard,WGUSA,43100.0,NaN,2017


In [16]:
# seperate studios so leaving in place and merging in hopes that nans match 
m_gross.loc[m_gross['title']=='Bluebeard'] 


,title,studio,domestic_gross,foreign_gross,year
317,Bluebeard,Strand,33500.0,5200.0,2010
3045,Bluebeard,WGUSA,43100.0,NaN,2017


In [17]:
#merged movie gross and movie budget
gross_budg_join =relevent_movies_10.merge(m_gross, left_on='movie',right_on='title',how ='inner')
gross_budg_join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1055 entries, 0 to 1054
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 1055 non-null   int64         
 1   release_date       1055 non-null   datetime64[ns]
 2   movie              1055 non-null   object        
 3   production_budget  1055 non-null   float64       
 4   domestic_gross_x   1055 non-null   float64       
 5   worldwide_gross    1055 non-null   float64       
 6   release_year       1055 non-null   int64         
 7   title              1055 non-null   object        
 8   studio             1054 non-null   object        
 9   domestic_gross_y   1054 non-null   float64       
 10  foreign_gross      896 non-null    float64       
 11  year               1055 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int64(3), object(3)
memory usage: 107.1+ KB


In [20]:
#Load in popular to properly join df to my gross_budg_join,
merged_2= gross_budg_join.merge(popular, left_on='title', right_on='title',how ='inner')
merged_2

,id_x,release_date_x,movie,production_budget,domestic_gross_x,worldwide_gross,release_year,title,studio,domestic_gross_y,foreign_gross,year,genre_ids,id_y,original_language,original_title,popularity,release_date_y,vote_average,vote_count
0,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,2011,Pirates of the Caribbean: On Stranger Tides,BV,241100000.0,804600000.0,2011,"[12, 28, 14]",1865,en,Pirates of the Caribbean: On Stranger Tides,30.579,2011-05-20,6.4,8571
1,4,2015-05-01,Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,2015,Avengers: Age of Ultron,BV,459000000.0,946400000.0,2015,"[28, 12, 878]",99861,en,Avengers: Age of Ultron,44.383,2015-05-01,7.3,13457
2,7,2018-04-27,Avengers: Infinity War,300000000.0,678815482.0,2.048134e+09,2018,Avengers: Infinity War,BV,678800000.0,NaN,2018,"[12, 28, 14]",299536,en,Avengers: Infinity War,80.773,2018-04-27,8.3,13948
3,9,2017-11-17,Justice League,300000000.0,229024295.0,6.559452e+08,2017,Justice League,WB,229000000.0,428900000.0,2017,"[28, 12, 14, 878]",141052,en,Justice League,34.953,2017-11-17,6.2,7510
4,9,2017-11-17,Justice League,300000000.0,229024295.0,6.559452e+08,2017,Justice League,WB,229000000.0,428900000.0,2017,"[28, 12, 14, 878]",141052,en,Justice League,34.953,2017-11-17,6.2,7510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,48,2011-07-22,Another Earth,175000.0,1321194.0,2.102779e+06,2011,Another Earth,FoxS,1300000.0,456000.0,2011,"[18, 878]",55420,en,Another Earth,10.030,2011-07-22,6.7,853
1209,68,2012-04-27,Sound of My Voice,135000.0,408015.0,4.294480e+05,2012,Sound of My Voice,FoxS,408000.0,NaN,2012,"[878, 18, 9648, 53]",86812,en,Sound of My Voice,7.212,2011-01-22,6.4,169
1210,73,2012-06-15,Your Sister's Sister,120000.0,1597486.0,3.090593e+06,2012,Your Sister's Sister,IFC,1600000.0,NaN,2012,"[18, 35]",84355,en,Your Sister's Sister,7.115,2012-06-14,6.6,192
1211,80,2015-07-10,The Gallows,100000.0,22764410.0,4.165647e+07,2015,The Gallows,WB (NL),22800000.0,20200000.0,2015,"[27, 53]",299245,en,The Gallows,9.166,2015-07-10,4.8,591


In [21]:
#Cleaning merged_2 ..First dropping columns not needed
merged_2.drop(['domestic_gross_y','id_y','release_date_y'], axis=1, inplace = True)

In [23]:
merged_2[merged_2.duplicated(subset=['movie','year'])]

,id_x,release_date_x,movie,production_budget,domestic_gross_x,worldwide_gross,release_year,title,studio,foreign_gross,year,genre_ids,original_language,original_title,popularity,vote_average,vote_count
4,9,2017-11-17,Justice League,300000000.0,229024295.0,6.559452e+08,2017,Justice League,WB,428900000.0,2017,"[28, 12, 14, 878]",en,Justice League,34.953,6.2,7510
6,10,2015-11-06,Spectre,300000000.0,200074175.0,8.796209e+08,2015,Spectre,Sony,680600000.0,2015,"[28, 12, 80]",en,Spectre,30.318,6.4,6719
25,42,2018-02-16,Black Panther,200000000.0,700059566.0,1.348258e+09,2018,Black Panther,BV,646900000.0,2018,"[28, 12, 14, 878]",en,Black Panther,44.140,7.4,12365
28,45,2016-12-16,Rogue One: A Star Wars Story,200000000.0,532177324.0,1.049103e+09,2016,Rogue One: A Star Wars Story,BV,523900000.0,2016,"[28, 12, 878]",en,Rogue One: A Star Wars Story,21.401,7.5,9296
45,73,2015-11-25,The Good Dinosaur,187500000.0,123087120.0,3.337710e+08,2015,The Good Dinosaur,BV,209100000.0,2015,"[12, 16, 10751]",en,The Good Dinosaur,17.675,6.7,3135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155,94,2015-03-13,It Follows,2000000.0,14674077.0,2.325076e+07,2015,It Follows,RTWC,NaN,2015,"[27, 53]",en,It Follows,18.419,6.5,3485
1175,100,2013-10-25,The Square,1500000.0,124244.0,1.762620e+05,2013,The Square,Magn.,NaN,2017,[18],en,The Square,9.396,6.9,606
1190,58,2014-05-09,Palo Alto,1000000.0,767732.0,1.156309e+06,2014,Palo Alto,Trib.,NaN,2014,[18],en,Palo Alto,7.262,6.4,528
1192,62,2011-07-29,The Future,1000000.0,568662.0,1.239174e+06,2011,The Future,RAtt.,NaN,2011,[18],it,Il futuro,2.336,5.8,11


In [24]:
#merged_2.to_csv('/Users/carlosgovea/Documents/Flatiron/DSNYC103122/Phase-1-project-template/notebooks/merged_2.csv')

In [26]:
#SQL LOAD AND CLEANING NOARRRRRRR.. LOAD IN AND THEN # TO STOP LONG WAITING 

sq = sqlite3.connect('/Users/carlosgovea/Documents/Flatiron/DSNYC103122/Phase-1-project-template/zippedData/im.db')
cur = sq.cursor() # Used to execute SQL commands
cur.execute("""SELECT name FROM sqlite_master WHERE type = 'table';""")
#sql_movie_basics = pd.read_sql_query("SELECT * from movie_basics", sq)
#sql_directors = pd.read_sql_query("SELECT * from directors", sq)
#sql_known_for = pd.read_sql_query("SELECT * from known_for", sq) #YOU WONT NEED THIS...I ENDED UP NEEDING YOU 
#sql_movie_akas = pd.read_sql_query("SELECT * from movie_akas", sq) #YOU WONT NEED THIS... I ENDED UP NEEDING YOU TOO
#sql_movie_ratings = pd.read_sql_query("SELECT * from movie_ratings", sq)
#sql_persons = pd.read_sql_query("SELECT * from persons", sq) 
#sql_principals = pd.read_sql_query("SELECT * from principals", sq)
#sql_writers = pd.read_sql_query("SELECT * from writers", sq)
#sql_movie_ratings.to_csv('/Users/carlosgovea/Documents/Flatiron/DSNYC103122/Phase-1-project-template/zippedData/movie_ratings.csv')
#sql_movie_basics.to_csv('/Users/carlosgovea/Documents/Flatiron/DSNYC103122/Phase-1-project-template/zippedData/movie_basics.csv')
sql_principals

,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0111414,2,nm0398271,director,None,None
2,tt0111414,3,nm3739909,producer,producer,None
3,tt0323808,10,nm0059247,editor,None,None
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"
...,...,...,...,...,...,...
1028181,tt9692684,1,nm0186469,actor,None,"[""Ebenezer Scrooge""]"
1028182,tt9692684,2,nm4929530,self,None,"[""Herself"",""Regan""]"
1028183,tt9692684,3,nm10441594,director,None,None
1028184,tt9692684,4,nm6009913,writer,writer,None


In [27]:
merged_2

,id_x,release_date_x,movie,production_budget,domestic_gross_x,worldwide_gross,release_year,title,studio,foreign_gross,year,genre_ids,original_language,original_title,popularity,vote_average,vote_count
0,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,2011,Pirates of the Caribbean: On Stranger Tides,BV,804600000.0,2011,"[12, 28, 14]",en,Pirates of the Caribbean: On Stranger Tides,30.579,6.4,8571
1,4,2015-05-01,Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,2015,Avengers: Age of Ultron,BV,946400000.0,2015,"[28, 12, 878]",en,Avengers: Age of Ultron,44.383,7.3,13457
2,7,2018-04-27,Avengers: Infinity War,300000000.0,678815482.0,2.048134e+09,2018,Avengers: Infinity War,BV,NaN,2018,"[12, 28, 14]",en,Avengers: Infinity War,80.773,8.3,13948
3,9,2017-11-17,Justice League,300000000.0,229024295.0,6.559452e+08,2017,Justice League,WB,428900000.0,2017,"[28, 12, 14, 878]",en,Justice League,34.953,6.2,7510
4,9,2017-11-17,Justice League,300000000.0,229024295.0,6.559452e+08,2017,Justice League,WB,428900000.0,2017,"[28, 12, 14, 878]",en,Justice League,34.953,6.2,7510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,48,2011-07-22,Another Earth,175000.0,1321194.0,2.102779e+06,2011,Another Earth,FoxS,456000.0,2011,"[18, 878]",en,Another Earth,10.030,6.7,853
1209,68,2012-04-27,Sound of My Voice,135000.0,408015.0,4.294480e+05,2012,Sound of My Voice,FoxS,NaN,2012,"[878, 18, 9648, 53]",en,Sound of My Voice,7.212,6.4,169
1210,73,2012-06-15,Your Sister's Sister,120000.0,1597486.0,3.090593e+06,2012,Your Sister's Sister,IFC,NaN,2012,"[18, 35]",en,Your Sister's Sister,7.115,6.6,192
1211,80,2015-07-10,The Gallows,100000.0,22764410.0,4.165647e+07,2015,The Gallows,WB (NL),20200000.0,2015,"[27, 53]",en,The Gallows,9.166,4.8,591


In [28]:
sql_merged = sql_movie_basics.merge(sql_movie_ratings, left_on ='movie_id', right_on='movie_id', how ='outer')

In [29]:
sql_person_match = sql_persons.merge(sql_writers,left_on='person_id', right_on='person_id', how ='inner')

In [30]:
sql_person_match

,person_id,primary_name,birth_year,death_year,primary_profession,movie_id
0,nm0064023,Bryan Beasley,NaN,NaN,"director,producer,writer",tt3501180
1,nm0065847,Michael Frost Beckner,1963.0,NaN,"writer,producer,miscellaneous",tt6349302
2,nm0068874,Hava Kohav Beller,NaN,NaN,"director,writer,producer",tt7701650
3,nm0070482,Joel Bender,1948.0,NaN,"editor,director,writer",tt3790232
4,nm0070482,Joel Bender,1948.0,NaN,"editor,director,writer",tt3790232
...,...,...,...,...,...,...
255866,nm9980896,Rama Narayanan,NaN,NaN,"director,writer",tt8715016
255867,nm9981679,Samir Eshra,NaN,NaN,"director,writer,cinematographer",tt8717234
255868,nm9993380,Pegasus Envoyé,NaN,NaN,"director,actor,writer",tt8743182
255869,nm9993380,Pegasus Envoyé,NaN,NaN,"director,actor,writer",tt8743182


In [31]:
#DONT NEED YOU 
sql_person_match.drop(['primary_profession','birth_year','death_year'], axis=1,inplace=True)

In [32]:
#TRYING TO FILTER OUT DIRECTOR
all_merge = merged_2.merge(sql_merged,left_on='title',right_on='primary_title',how='inner')

In [33]:
all_merge

,id_x,release_date_x,movie,production_budget,domestic_gross_x,worldwide_gross,release_year,title,studio,foreign_gross,...,vote_average,vote_count,movie_id,primary_title,original_title_y,start_year,runtime_minutes,genres,averagerating,numvotes
0,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,2011,Pirates of the Caribbean: On Stranger Tides,BV,804600000.0,...,6.4,8571,tt1298650,Pirates of the Caribbean: On Stranger Tides,Pirates of the Caribbean: On Stranger Tides,2011,136.0,"Action,Adventure,Fantasy",6.6,447624.0
1,4,2015-05-01,Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,2015,Avengers: Age of Ultron,BV,946400000.0,...,7.3,13457,tt2395427,Avengers: Age of Ultron,Avengers: Age of Ultron,2015,141.0,"Action,Adventure,Sci-Fi",7.3,665594.0
2,7,2018-04-27,Avengers: Infinity War,300000000.0,678815482.0,2.048134e+09,2018,Avengers: Infinity War,BV,NaN,...,8.3,13948,tt4154756,Avengers: Infinity War,Avengers: Infinity War,2018,149.0,"Action,Adventure,Sci-Fi",8.5,670926.0
3,9,2017-11-17,Justice League,300000000.0,229024295.0,6.559452e+08,2017,Justice League,WB,428900000.0,...,6.2,7510,tt0974015,Justice League,Justice League,2017,120.0,"Action,Adventure,Fantasy",6.5,329135.0
4,9,2017-11-17,Justice League,300000000.0,229024295.0,6.559452e+08,2017,Justice League,WB,428900000.0,...,6.2,7510,tt0974015,Justice League,Justice League,2017,120.0,"Action,Adventure,Fantasy",6.5,329135.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,48,2011-07-22,Another Earth,175000.0,1321194.0,2.102779e+06,2011,Another Earth,FoxS,456000.0,...,6.7,853,tt1549572,Another Earth,Another Earth,2011,92.0,"Drama,Romance,Sci-Fi",7.0,85839.0
1796,68,2012-04-27,Sound of My Voice,135000.0,408015.0,4.294480e+05,2012,Sound of My Voice,FoxS,NaN,...,6.4,169,tt1748207,Sound of My Voice,Sound of My Voice,2011,85.0,"Drama,Mystery,Sci-Fi",6.6,19387.0
1797,73,2012-06-15,Your Sister's Sister,120000.0,1597486.0,3.090593e+06,2012,Your Sister's Sister,IFC,NaN,...,6.6,192,tt1742336,Your Sister's Sister,Your Sister's Sister,2011,90.0,"Comedy,Drama",6.7,24780.0
1798,80,2015-07-10,The Gallows,100000.0,22764410.0,4.165647e+07,2015,The Gallows,WB (NL),20200000.0,...,4.8,591,tt2309260,The Gallows,The Gallows,2015,81.0,"Horror,Mystery,Thriller",4.2,17763.0


In [34]:
all_merge.drop(['start_year','original_title_y','primary_title','vote_count','vote_average','original_title_x','original_language'], axis=1,inplace=True)

In [39]:
sql_person_match = sql_persons.merge(sql_writers,left_on='person_id', right_on='person_id', how ='inner')

In [40]:
sql_person_match
sql_person_match.drop(['primary_profession','birth_year','death_year'], axis=1,inplace=True)
sql_person_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255871 entries, 0 to 255870
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   person_id     255871 non-null  object
 1   primary_name  255871 non-null  object
 2   movie_id      255871 non-null  object
dtypes: object(3)
memory usage: 7.8+ MB


In [41]:
all_merge = merged_2.merge(sql_merged,left_on='title',right_on='primary_title',how='inner')

In [42]:
all_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1800 entries, 0 to 1799
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id_x               1800 non-null   int64         
 1   release_date_x     1800 non-null   datetime64[ns]
 2   movie              1800 non-null   object        
 3   production_budget  1800 non-null   float64       
 4   domestic_gross_x   1800 non-null   float64       
 5   worldwide_gross    1800 non-null   float64       
 6   release_year       1800 non-null   int64         
 7   title              1800 non-null   object        
 8   studio             1800 non-null   object        
 9   foreign_gross      1477 non-null   float64       
 10  year               1800 non-null   int64         
 11  genre_ids          1800 non-null   object        
 12  original_language  1800 non-null   object        
 13  original_title_x   1800 non-null   object        
 14  populari

In [43]:
all_merge.drop(['start_year','original_title_y','primary_title','vote_count','vote_average','original_title_x','original_language'], axis=1,inplace=True)

In [47]:
sql_person_match = sql_persons.merge(sql_writers,left_on='person_id', right_on='person_id', how ='inner')

In [48]:
sql_movie_match = sql_merged.merge(sql_person, left_on = 'movie_id', right_on ='movie_id', how='inner')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255871 entries, 0 to 255870
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   person_id     255871 non-null  object
 1   primary_name  255871 non-null  object
 2   movie_id      255871 non-null  object
dtypes: object(3)
memory usage: 7.8+ MB


In [52]:
sql_movie_match['types'].value_counts()

In [61]:
#String contains function to filter out only directors
sql_person_match.info() 
sql_person_match['directors_dummy']= sql_person_match['primary_profession'].str.contains('director')
sql_person_match =sql_person_match.loc[sql_person_match['directors_dummy']==True]
sql_person_match.drop(['birth_year','death_year'], axis=1,inplace=True)
sql_person_match.dropna()

In [62]:

sql_person_cat =sql_person_match.merge(sql_principals, left_on ='person_id', right_on='person_id',how='inner')

In [63]:
sql_person_cat = sql_person_cat.loc[sql_person_cat['category']=='director']

In [69]:
sql_person_cat.drop(['movie_id_y','job','characters',],axis=1,inplace=True)

In [66]:
sql_person_cat.drop(['primary_profession'],axis=1, inplace=True)

In [67]:
sql_person_cat.drop(['ordering'],axis=1, inplace=True)

In [70]:
sql_person_cat

,person_id,primary_name,movie_id_x,category
0,nm0064023,Bryan Beasley,tt3501180,director
1,nm0064023,Bryan Beasley,tt3501180,director
3,nm0068874,Hava Kohav Beller,tt7701650,director
4,nm0070482,Joel Bender,tt3790232,director
5,nm0070482,Joel Bender,tt3790232,director
...,...,...,...,...
835032,nm9980896,Rama Narayanan,tt8715016,director
835033,nm9981679,Samir Eshra,tt8717234,director
835034,nm9993380,Pegasus Envoyé,tt8743182,director
835035,nm9993380,Pegasus Envoyé,tt8743182,director


In [78]:
all_merged = pd.read_csv('/Users/carlosgovea/Documents/Flatiron/DSNYC103122/Phase-1-project-template/zippedData/all_merged.csv')

In [72]:
all_merged_cat = all_merged.merge(sql_person_cat,left_on='movie_id',right_on='movie_id_x',how='outer')

In [73]:
all_merged_cat.dropna()

In [74]:
all_merged_cat.drop_duplicates(subset=['movie'], keep='first',inplace =True)

In [75]:
#COOL FUNCTION TO CATEGORIZE VALUES IN N BINS AND NAMIING THEM 
all_merged_cat['profitability'] =pd.qcut(all_merged_cat['profit_rate'], q=3, labels=['Low','Moderate','High'])

In [76]:
all_merged_cat.dropna()

In [77]:
all_merged_cat['primary_name'].value_counts()

In [79]:
# ONLY NEED THOSE WHO HAVE HIGH PROFIT RATES
all_merged_cat = all_merged_cat.loc[all_merged_cat['profitability']=='High',:]

In [80]:
all_merged_cat.drop(['release_date','year','genre_ids','genres','person_id','movie_id_x'],axis=1, inplace=True)

In [81]:
all_merged_cat = all_merged_cat.groupby(['studio', 'primary_name','primary_share']).sum()

In [82]:
all_merged_cat = all_merged_cat.sort_values(by='popularity',ascending=False)

In [83]:
#MULTI-INDEXED GROUOPING 
grouped_cat = all_merged_cat.groupby(level =[0])['profit'].transform(max)
grouped_cat.drop_duplicates(keep='first',inplace=True)
grouped_cat = grouped_cat.loc[['Uni.','Fox','BV','Par.','Sony','WB (NL)','WB','Wein.','FoxS','LGF'],:]
grouped_cat.sort_values(ascending=False, inplace =True)
grouped_cat.to_csv('/Users/carlosgovea/Documents/Flatiron/DSNYC103122/Phase-1-project-template/zippedData/director_profit.csv')